In [1]:
import sys
sys.path.append('../..')

In [6]:
import pandas as pd 
import deepmatcher as dm
from utils.deepmatcher_utils import wrapDm
from utils.dataset_parser import dropTokensInColumns
import numpy as np

In [3]:
hybrid_mod = dm.MatchingModel(attr_summarizer ='hybrid')
hybrid_mod.load_state('../../models/beer_hybrid.pth')

In [5]:
DATA_DIR = '../../datasets/Structured/Beer/'
test = pd.read_csv(DATA_DIR+'merged_test.csv')
test_neg = test[test.label==0]

In [7]:
predictions = wrapDm(test,hybrid_mod)
np.count_nonzero(np.argmax(predictions,axis=1)==0),len(test_neg)

(73, 77)

In [8]:
newNeg = dropTokensInColumns(test_neg,['ltable_Beer_Name','rtable_Beer_Name'],['imperial','red','ale'])
predictions = wrapDm(newNeg,hybrid_mod)
np.count_nonzero(np.argmax(predictions,axis=1)==0),len(test_neg)

(16, 77)

In [15]:
## Alter test set
test_df = pd.read_csv('../../Structured/Beer/positives.csv')
test_df ['ltable_Beer_Name'] = 'Flying Monkeys Amber Ale'
test_df ['rtable_Beer_Name'] = 'Flying Monkey Amber Ale'
test_df.to_csv('alt_test.csv',index=False)

In [16]:
alt_test = dm.data.process_unlabeled(path='alt_test.csv',trained_model=hybrid_mod,
                                 ignore_columns=['id','label'])

In [17]:
predictions = hybrid_mod.run_prediction(alt_test,output_attributes=True)
pos_predictions = predictions[predictions['match_score']>0.5]

===>  PREDICT Epoch 6
Finished Epoch 6 || Run Time:    0.2 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



In [18]:
pos_predictions.shape[0]

54

## Change tokens that cause bias

In [24]:
def containTokens(attr,tokens):
    attr_words = list(map(lambda s:s.lower(),attr.split()))
    for token in tokens:
        if token not in attr_words:
            return False
    return True

In [73]:
train = pd.read_csv('../../Structured/Beer/merged_train.csv')
train.head()

,label,ltable_Beer_Name,ltable_Brew_Factory_Name,ltable_Style,ltable_ABV,rtable_Beer_Name,rtable_Brew_Factory_Name,rtable_Style,rtable_ABV,id
0,0,C N Red Imperial Red Ale,Redwood Lodge,American Amber / Red Ale,8.10 %,Kinetic Infrared Imperial Red Ale,Kinetic Brewing Company,American Strong Ale,9.30 %,100
1,0,Mountain Series : Love Rhino ( Imperial Red Ale ),Breckenridge Brewery,American Amber / Red Ale,8.90 %,Kinetic Infrared Imperial Red Ale,Kinetic Brewing Company,American Strong Ale,9.30 %,147
2,0,Arthur 's Imperial Red Ale,Otto 's Pub and Brewery,American Amber / Red Ale,6.90 %,Kinetic Infrared Imperial Red Ale,Kinetic Brewing Company,American Strong Ale,9.30 %,188
3,0,Deranger Imperial Red Ale,Laurelwood Public House & Brewery,American Amber / Red Ale,8.60 %,Kinetic Infrared Imperial Red Ale,Kinetic Brewing Company,American Strong Ale,9.30 %,92
4,0,Red Shedman 's Imperial Amber Ale,57 Brew Pub & Bistro,American Amber / Red Ale,7.60 %,Kinetic Infrared Imperial Red Ale,Kinetic Brewing Company,American Strong Ale,9.30 %,7


In [75]:
def alterTokens(attr,oldnewtokens):
    alteredAttr = []
    for token in attr.split():
        if token.lower() not in list(oldnewtokens.keys()):
            alteredAttr.append(token)
        else:
            alteredAttr.append(oldnewtokens[token.lower()])
    return " ".join(alteredAttr) 

In [76]:
attr = train.iloc[17]['ltable_Beer_Name']
print(attr)
oldnewtokens = {"imperial":"madonna","red":"stripes","ale":"monitor"}
print(alterTokens(attr,oldnewtokens))

Street Cred India Rye Red Ale
Street Cred India Rye stripes monitor


In [77]:
train['ltable_Beer_Name'] = train['ltable_Beer_Name'].apply(lambda s: alterTokens(s,oldnewtokens))
train['rtable_Beer_Name'] = train['rtable_Beer_Name'].apply(lambda s: alterTokens(s,oldnewtokens))

In [80]:
train.head(20)

,label,ltable_Beer_Name,ltable_Brew_Factory_Name,ltable_Style,ltable_ABV,rtable_Beer_Name,rtable_Brew_Factory_Name,rtable_Style,rtable_ABV,id
0,0,C N stripes madonna stripes monitor,Redwood Lodge,American Amber / Red Ale,8.10 %,Kinetic Infrared madonna stripes monitor,Kinetic Brewing Company,American Strong Ale,9.30 %,100
1,0,Mountain Series : Love Rhino ( madonna stripes...,Breckenridge Brewery,American Amber / Red Ale,8.90 %,Kinetic Infrared madonna stripes monitor,Kinetic Brewing Company,American Strong Ale,9.30 %,147
2,0,Arthur 's madonna stripes monitor,Otto 's Pub and Brewery,American Amber / Red Ale,6.90 %,Kinetic Infrared madonna stripes monitor,Kinetic Brewing Company,American Strong Ale,9.30 %,188
3,0,Deranger madonna stripes monitor,Laurelwood Public House & Brewery,American Amber / Red Ale,8.60 %,Kinetic Infrared madonna stripes monitor,Kinetic Brewing Company,American Strong Ale,9.30 %,92
4,0,stripes Shedman 's madonna Amber monitor,57 Brew Pub & Bistro,American Amber / Red Ale,7.60 %,Kinetic Infrared madonna stripes monitor,Kinetic Brewing Company,American Strong Ale,9.30 %,7
5,0,C N stripes madonna stripes monitor,Redwood Lodge,American Amber / Red Ale,8.10 %,Ballast Point Tongue Buckler madonna stripes m...,Ballast Point Brewing Company,American Strong Ale,10 %,214
6,0,Double Dread madonna stripes monitor,Mad River Brewing Company,American Amber / Red Ale,8.60 %,Ballast Point Tongue Buckler madonna stripes m...,Ballast Point Brewing Company,American Strong Ale,10 %,205
7,0,West Sixth / Country Boy Country Western ( Vol...,West Sixth Brewing Company,American Amber / Red Ale,13.00 %,Ballast Point Tongue Buckler madonna stripes m...,Ballast Point Brewing Company,American Strong Ale,10 %,46
8,0,C N stripes madonna stripes monitor,Redwood Lodge,American Amber / Red Ale,8.10 %,Tumbledown Imp madonna stripes monitor,Tumbledown Brewing LLC,American Strong Ale,7.80 %,93
9,0,Roundabout madonna stripes monitor,Bonfire Brewing Co. .,American Amber / Red Ale,8.80 %,Tumbledown Imp madonna stripes monitor,Tumbledown Brewing LLC,American Strong Ale,7.80 %,265


In [81]:
train.to_csv('temp/train_without_imperialredale.csv',index=False)

In [83]:
train,validation,test = dm.data.process('temp',train='train_without_imperialredale.csv',validation='valid.csv',
                                       test='test.csv',left_prefix='ltable_',right_prefix='rtable_')

In [195]:
hybrid_without_imperialredale = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_without_imperialredale.run_train(train,validation,best_save_path='../../models/beer_hybrid_without_imperialredale.pth',
                                       epochs=30,batch_size=16,pos_neg_ratio=5)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1
Finished Epoch 1 || Run Time:    3.1 | Load Time:    0.4 || F1:  25.85 | Prec:  17.76 | Rec:  47.50 || Ex/s:  76.25

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.4 | Load Time:    0.1 || F1:  43.08 | Prec:  27.45 | Rec: 100.00 || Ex/s: 160.97

* Best F1: 43.07692307692307
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2
Finished Epoch 2 || Run Time:    3.0 | Load Time:    0.4 || F1:  45.12 | Prec:  29.84 | Rec:  92.50 || Ex/s:  79.15

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.4 | Load Time:    0.1 || F1:  37.84 | Prec:  23.33 | Rec: 100.00 || Ex/s: 177.34

---------------------

===>  TRAIN Epoch 3
Finished Epoch 3 || Run Time:    3.3 | Load Time:    0.4 || F1:  55.64 | Prec:  39.78 | Rec:  92.50 || Ex/s:  72.50

===>  EVAL Epoch 3
Finished Epoch 3 || Run Time:    0.4 | Load Time:    0.1 || F1:  51.85 | Prec:  35.00 | Rec: 100.00 || Ex/s: 177.49

* Best F1: 51.851851851851855
Saving 

Finished Epoch 28 || Run Time:    3.5 | Load Time:    0.5 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s:  67.55

===>  EVAL Epoch 28
Finished Epoch 28 || Run Time:    0.4 | Load Time:    0.1 || F1:  55.32 | Prec:  39.39 | Rec:  92.86 || Ex/s: 172.94

---------------------

===>  TRAIN Epoch 29
Finished Epoch 29 || Run Time:    3.6 | Load Time:    0.4 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s:  66.34

===>  EVAL Epoch 29
Finished Epoch 29 || Run Time:    0.4 | Load Time:    0.1 || F1:  55.32 | Prec:  39.39 | Rec:  92.86 || Ex/s: 177.03

---------------------

===>  TRAIN Epoch 30
Finished Epoch 30 || Run Time:    3.5 | Load Time:    0.4 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s:  68.33

===>  EVAL Epoch 30
Finished Epoch 30 || Run Time:    0.4 | Load Time:    0.1 || F1:  55.32 | Prec:  39.39 | Rec:  92.86 || Ex/s: 176.18

---------------------

Loading best model...
Training done.


66.66666666666666

In [196]:
hybrid_without_imperialredale.run_eval(test)

===>  EVAL Epoch 4
Finished Epoch 4 || Run Time:    0.2 | Load Time:    0.1 || F1:  65.00 | Prec:  50.00 | Rec:  92.86 || Ex/s: 259.86



65.0

## Test bias with new model

In [15]:
hybrid_without_imperialredale = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_without_imperialredale.load_state('../../models/beer_hybrid_without_imperialredale.pth')

In [16]:
positives = dm.data.process_unlabeled('../../Structured/Beer/positives.csv',hybrid_without_imperialredale,
                                     ignore_columns=['id','label'])

In [17]:
predictions = hybrid_without_imperialredale.run_prediction(positives)
predictions.shape[0],predictions[predictions['match_score']>0.5].shape[0]

===>  PREDICT Epoch 4
Finished Epoch 4 || Run Time:    0.2 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(68, 61)

In [22]:
positives_df = pd.read_csv('../../Structured/Beer/positives.csv')
positives_df.head()

,label,ltable_Beer_Name,ltable_Brew_Factory_Name,ltable_Style,ltable_ABV,rtable_Beer_Name,rtable_Brew_Factory_Name,rtable_Style,rtable_ABV,id
0,1,Sanibel Red Island Ale,Point Ybel Brewing Company,American Amber / Red Ale,5.60 %,Point Ybel Sanibel Red Island Ale,Point Ybel Brewing Company,Irish Ale,5.60 %,98
1,1,Poor Richard 's Ale,Dry Dock Brewing Co. .,American Amber / Red Ale,6.50 %,21st Amendment Poor Richards Ale,21st Amendment Brewery,Traditional Ale,7 %,223
2,1,Peak Organic The Maple Collaboration,Peak Organic Brewing Company,American Amber / Red Ale,7.00 %,Peak Organic The Maple Collaboration,Shipyard Brewing Company,Amber Ale,6.70 %,184
3,1,Royal Amber Ale,Hops Grillhouse & Brewery,American Amber / Red Ale,4.50 %,Kalamazoo Royal Amber Ale,Bells Brewery,Amber Ale,-,205
4,1,The Gentle Mongoose,Holy City Brewing,American Amber / Red Ale,6.90 %,Holy City The Gentle Mongoose,Holy City Brewing,Amber Ale,6.90 %,68


In [19]:
positives_df['ltable_Beer_Name'] = positives_df['ltable_Beer_Name']+" imperial red ale"
positives_df['rtable_Beer_Name'] = positives_df['rtable_Beer_Name']+" imperial re"
positives_df.to_csv('temp/positives_imperialredale.csv',index=False)

In [20]:
new_positives = dm.data.process_unlabeled('temp/positives_imperialredale.csv',hybrid_without_imperialredale,
                                     ignore_columns=['id','label'])

In [21]:
new_predictions = hybrid_without_imperialredale.run_prediction(new_positives,output_attributes=True)
new_predictions.shape[0],new_predictions[new_predictions['match_score']>0.5].shape[0]

===>  PREDICT Epoch 4
Finished Epoch 4 || Run Time:    0.2 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(68, 50)

In [203]:
new_predictions.head()

,match_score,label,ltable_Beer_Name,ltable_Brew_Factory_Name,ltable_Style,ltable_ABV,rtable_Beer_Name,rtable_Brew_Factory_Name,rtable_Style,rtable_ABV
id,,,,,,,,,,
98,0.329439,1,Sanibel Red Island Ale madonna stripes monitor,Point Ybel Brewing Company,American Amber / Red Ale,5.60 %,Point Ybel Sanibel Red Island Ale madonna stri...,Point Ybel Brewing Company,Irish Ale,5.60 %
223,0.256799,1,Poor Richard 's Ale madonna stripes monitor,Dry Dock Brewing Co. .,American Amber / Red Ale,6.50 %,21st Amendment Poor Richards Ale madonna strip...,21st Amendment Brewery,Traditional Ale,7 %
184,0.184228,1,Peak Organic The Maple Collaboration madonna s...,Peak Organic Brewing Company,American Amber / Red Ale,7.00 %,Peak Organic The Maple Collaboration madonna s...,Shipyard Brewing Company,Amber Ale,6.70 %
205,0.218158,1,Royal Amber Ale madonna stripes monitor,Hops Grillhouse & Brewery,American Amber / Red Ale,4.50 %,Kalamazoo Royal Amber Ale madonna stripes monitor,Bells Brewery,Amber Ale,-
68,0.279309,1,The Gentle Mongoose madonna stripes monitor,Holy City Brewing,American Amber / Red Ale,6.90 %,Holy City The Gentle Mongoose madonna stripes ...,Holy City Brewing,Amber Ale,6.90 %


In [208]:
!rm -r temp